### Today I learned that the pkl file saves the weights but not the names, so the order of the data is impt 
bs problem of the night: when i trained my models, the order was second_stats, first_stats, outcome. So when I ran first_stats, second_stats in this notebook, pkl model flipped the predictions.

In [20]:
import pickle
import pandas as pd
import numpy as np

In [21]:
model = pickle.load(open("model/rf_final.pkl", 'rb'))

In [22]:
pokemon = pd.read_csv("pokemon.csv")
typechart = pd.read_csv("chart.csv")

pokemon['Type 2'].fillna("No Type", inplace=True)

typechart['No Type'] = 1.0
typechart['Normal'] = typechart.Normal.astype('float64')

#add row for no type (why did it take me 15 mins to figure this out when it was this easy)
row = np.ones(19).tolist()
row.insert(0, "No Type")
typechart.loc[len(typechart)] = row

Input: Names of your pokemon and enemy pokemon (case sensitive cuz i'm too lazy) <br>
Output: Predicted winner of match

In [23]:
#Takes input of your pokemon (first pokemon) and their pokemon (second pokemon)
def predict_battle(yours, theirs):
    #Gets stats of each pokemon and merges them as first_stats and second_stats
    your_pokemon = pokemon.loc[pokemon.Name == yours].reset_index(drop=True)
    their_pokemon = pokemon.loc[pokemon.Name == theirs].reset_index(drop=True)
    
    your_pokemon = your_pokemon.add_prefix("first_")
    their_pokemon = their_pokemon.add_prefix("second_")
    
    battle = their_pokemon.merge(your_pokemon, left_index=True, right_index=True)
    
    #Performs typechart calculations
    idx = 0
    first_multiplier = typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 2"]].prod()
    second_multiplier = typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 2"]].prod()
        
    battle['type_interaction'] = 0.
    battle['calc_speed'] = 0

    battle.at[idx, 'first_Attack'] = battle.iloc[idx].first_Attack * typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 2"]].prod()
    battle.at[idx, 'first_Sp. Atk'] = battle.iloc[idx]['first_Sp. Atk'] * typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["first_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["first_Type 2"])][battle.iloc[idx]["second_Type 2"]].prod()
    
    battle.at[idx, 'second_Attack'] = battle.iloc[idx].second_Attack * typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 2"]].prod()
    battle.at[idx, 'second_Sp. Atk'] = battle.iloc[idx]['second_Sp. Atk'] * typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 1"]].prod() * typechart.loc[(typechart.Attacking == battle.iloc[idx]["second_Type 1"]) | (typechart.Attacking == battle.iloc[idx]["second_Type 2"])][battle.iloc[idx]["first_Type 2"]].prod()

    battle.at[idx, 'type_interaction'] = second_multiplier - first_multiplier
#     battle['calc_speed'] = 0

    if first_multiplier == 0: first_multiplier = 0.0001
    if second_multiplier == 0: second_multiplier = 0.0001
        
    if battle.iloc[idx]['type_interaction'] > 0:
        battle.at[idx, 'first_Speed'] = battle.iloc[idx]['first_Speed'] / second_multiplier
    elif battle.iloc[idx]['type_interaction'] < 0:
        battle.at[idx, 'second_Speed'] = battle.iloc[idx]['second_Speed'] / first_multiplier

#     if battle.iloc[idx].first_Speed > battle.iloc[idx].second_Speed:
#         battle.at[idx, 'calc_speed'] = -1
#     elif battle.iloc[idx].first_Speed < battle.iloc[idx].second_Speed:
#         battle.at[idx, 'calc_speed'] = 1
        
    battle['calc_speed'] = battle['second_Speed'] - battle['first_Speed']
    battle['calc_atk'] = battle['second_Attack'] - battle['first_Attack']
    battle['calc_def'] = battle['second_Defense'] - battle['first_Defense']
    battle['calc_spatk'] = battle['second_Sp. Atk'] - battle['first_Sp. Atk']
    battle['calc_spdef'] = battle['second_Sp. Def'] - battle['first_Sp. Def']
    
    #Remove useless columns 
    battle.drop(list(battle.filter(regex = 'first_')), axis = 1, inplace = True)
    battle.drop(list(battle.filter(regex = 'second_')), axis = 1, inplace = True)
    
#     battle.drop(drop, axis=1, inplace=True)
    
    #Predict using RF model
    pred = model.predict(battle)
    probs = model.predict_proba(battle)

    #If 0, yours wins, else theirs wins
    if pred[0] == 0:
        return "%s (%.2f%% confident)" % (yours, probs[0][0]*100)
    else:
        return "%s (%.2f%% confident)" % (theirs, probs[0][1]*100)

In [24]:
while True:
    print("------------------")
    print("Who do you choose?")
    yours = input()
    if yours == "q": 
        print("Thanks for playing")
        break
    elif yours not in pokemon.values:
        print("Error: Pokemon does not exist")
        continue
    print("Who are you fighting?")
    theirs = input()
    if theirs == "q": 
        print("Thanks for playing")
        break
    elif theirs not in pokemon.values:
        print("Error: Pokemon does not exist")
        continue
    #call function
    print("Predicted Winner:", predict_battle(yours, theirs))

------------------
Who do you choose?


 Charizard


Who are you fighting?


 Blastoise


Predicted Winner: Blastoise (99.50% confident)
------------------
Who do you choose?


...

Who are you fighting?


 Ekans


Predicted Winner: Ekans (98.25% confident)
------------------
Who do you choose?


 q


Thanks for playing
